In [6]:
#installing libs
%pip install praw pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#importing required libs for scraping
import praw
import pandas as pd
from praw.models import MoreComments
import time
from datetime import datetime, timezone

In [2]:
#reddit creds
reddit = praw.Reddit(
    user_agent="Comment Extraction (by /u/lestergreeks)",
    client_id="YiK4KHJXxneFv0IiV8aOhg", 
    client_secret="8GfGLUx8E62B3sBUwJcie43RDBQm7A"
)

In [3]:
#using a reddit post
#https://www.reddit.com/r/IndiaSpeaks/comments/1fr7tyb/scrap_dealer_gifts_multiple_iphones_to_son_for/ - apple test
url = "https://www.reddit.com/r/IndiaSpeaks/comments/1fqhbsz/youtuber_ranveer_allahbadias_youtube_channel_got/"
submission = reddit.submission(url=url)

In [4]:
#getting all comments from a post
submission.comments.replace_more(limit=None)
posts = []
for top_level_comment in submission.comments.list():
    if isinstance(top_level_comment, MoreComments):
        continue
    if top_level_comment.author and "bot" not in top_level_comment.author.name.lower() and not top_level_comment.stickied:
        posts.append(top_level_comment.body)
posts = pd.DataFrame(posts,columns=["body"])
indexNames = posts[(posts.body == '[removed]') | (posts.body == '[deleted]')].index
posts.drop(indexNames, inplace=True)

In [5]:
posts

,body
0,I heard both his passwords where 1234567@ 💀
1,Hogya recover
2,It was the hackers who showed him the next doo...
3,"What this basically tells is that:\n\n""Life is..."
4,This is the exact reason why India should deve...
...,...
67,YouTube has decades worth of data plus backed ...
68,Yes they can join. But Youtube algo recommends...
69,1. This is completely unrelated to Ranbir's ch...
70,Lmao yoh are placing to much importance on tec...


In [33]:
#extracting all comments from top10 posts of a subreddit
subreddit = reddit.subreddit('technews')

all_comments = pd.DataFrame()

    

In [62]:
def extract_comments(submission, limit = 10):
    posts = []
    submission.comments.replace_more(limit=None)
    count = 0
    for top_level_comment in submission.comments.list():
        if isinstance(top_level_comment, MoreComments):
            continue
        if (top_level_comment.author and "bot" not in top_level_comment.author.name.lower() 
            and not top_level_comment.stickied):
            posts.append(top_level_comment.body.encode('utf-8', 'ignore').decode('utf-8'))
            count += 1
            
            # Break if we have reached the limit
            if count >= limit:
                break
    
    return posts


In [63]:
def process_posts():
    global all_comments
    attempt = 0
    for submission in subreddit.rising(limit=10):
        try:
            print(f"Processing post: {submission.title}")
            posts = extract_comments(submission, limit=10)
            posts_df = pd.DataFrame(posts, columns=["body"])
            indexNames = posts_df[(posts_df.body == '[removed]') | (posts_df.body == '[deleted]')].index
            posts_df.drop(indexNames, inplace=True)
            posts_df['post_title'] = submission.title
            posts_df['post_time'] = datetime.fromtimestamp(submission.created_utc, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

            all_comments = pd.concat([all_comments, posts_df], ignore_index=True)
        
        except praw.exceptions.RedditAPIException as e:
            if "RATELIMIT" in str(e):
                attempt += 1
                sleep_time = 120 * attempt
                print(f"Rate limit hit. Sleeping for {sleep_time} seconds. Error: {e}")
                time.sleep(sleep_time) 
                continue
            else:
                print(f"Encountered an error: {e}")
                continue
    
        time.sleep(60)


In [ ]:
process_posts()

Processing post: The Optimus robots at Tesla’s Cybercab event were humans in disguise


In [61]:
all_comments

,body,post_title,post_time
0,I was admittedly a bit uncomfortable when my A...,The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
1,"The Mechanical Turk, classic.",The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
2,"Teleoperated, so not quite Russian robots.",The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
3,I could see a transition phase where teleworke...,The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
4,These robotics aren’t even particularly ground...,The Optimus robots at Tesla’s Cybercab event w...,2024-10-13 18:08:39
...,...,...,...
1128,Chinese gang = CCP,OpenAI says Chinese gang tried to phish its staff,2024-10-12 01:31:55
1129,"Of course, they're a very high profile, high p...",OpenAI says Chinese gang tried to phish its staff,2024-10-12 01:31:55
1130,"Of course, only this time they noticed it",OpenAI says Chinese gang tried to phish its staff,2024-10-12 01:31:55
1131,And they did the same to every other company o...,OpenAI says Chinese gang tried to phish its staff,2024-10-12 01:31:55


In [53]:
all_comments.to_csv('red_data_2.csv', encoding='utf-8', index=False)